In [42]:
import numpy as np
from scipy.sparse import csr_matrix
import pandas as pd
from datetime import datetime
import os
import itertools
import math
from random import random

In [43]:
npzfile = np.load("../predictions.npz")

In [44]:
user = npzfile['arr_0'].tolist()
item = npzfile['arr_1'].tolist()
rating = npzfile['arr_2'].tolist()

In [84]:
user_movie = dict()
for i in range(len(user)):
    temp = [rating[i],item[i]]
    try :
        user_movie[user[i]].append(temp)
        user_movie[user[i]].sort(reverse=True)
    except KeyError :
        user_movie[user[i]] = list()
        user_movie[user[i]].append(temp)
    
user_movie['202']

[[4.346704398358171, '286'],
 [3.922339620784242, '604'],
 [3.5140981163559077, '318'],
 [3.383857121190669, '172'],
 [3.038413068601718, '484'],
 [2.9918176517359627, '173']]

In [53]:
file_name = "../input/u.data"
watched_movies = dict()
def parse_line(line):
    line = line.split("\t")
    uid, iid, r, timestamp = (line[i].strip() for i in range(4))
    return uid, iid, float(r), timestamp

def Read_Data(file_name,shuffle=True) :
    global watched_movies
    with open(os.path.expanduser(file_name)) as f:
        raw_ratings = [parse_line(line) for line in itertools.islice(f, 0, None)]
    if shuffle:
        np.random.seed(73)
        np.random.shuffle(raw_ratings)

    raw_len = len(raw_ratings)
    train = raw_ratings[:math.ceil(raw_len*0.8)]
    test = raw_ratings[math.ceil(raw_len*0.8):]
    for i in train :
        try :
            watched_movies[i[0]].append(i[1])
        except KeyError :
            watched_movies[i[0]] = list()
            watched_movies[i[0]].append(i[1])
            
            

In [54]:
Read_Data(file_name,True)

In [55]:
watched_movies['202']

['242',
 '172',
 '1',
 '604',
 '191',
 '258',
 '283',
 '481',
 '96',
 '286',
 '423',
 '173',
 '179']

In [139]:
f = "../input/u.item"
movie_dict = dict()
def Read_Data_pipe(file_name,shuffle=True) :
    global movie_dict
    global watched_movies
    with open(os.path.expanduser(file_name)) as f:
        raw_ratings = [parse_line_pipe(line) for line in itertools.islice(f, 0, None)]
    for i in range(len(raw_ratings)):
        movie_dict[raw_ratings[i][0]] = raw_ratings[i][1:]
    return raw_ratings
def parse_line_pipe(line):
    line = line.split("|")
    return line
mdata = Read_Data_pipe(f)

In [145]:
seen_by_user = dict()
pred_for_user = dict()
for k in user_movie :
    mov = user_movie[k]
    for i in mov:
        ele = i[1]
        if ele not in watched_movies[k]:
            m_predict = movie_dict[ele]
            try :
                pred_for_user[k].append(m_predict)
            except KeyError :
                pred_for_user[k] = list()
                pred_for_user[k].append(m_predict)                    
        else:
            m_seen = movie_dict[ele]
            try :
                seen_by_user[k].append(m_seen)
            except KeyError :
                seen_by_user[k] = list()
                seen_by_user[k].append(m_seen)

In [167]:
print((pred_for_user['23'])) # i is the item(movie) and j is the corresponding entry for that movie

[['Heathers (1989)', '01-Jan-1989', '', 'http://us.imdb.com/M/title-exact?Heathers%20(1989)', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0\n'], ['Raising Arizona (1987)', '01-Jan-1987', '', 'http://us.imdb.com/M/title-exact?Raising%20Arizona%20(1987)', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0\n'], ['Man Who Would Be King, The (1975)', '01-Jan-1975', '', 'http://us.imdb.com/M/title-exact?Man%20Who%20Would%20Be%20King,%20The%20(1975)', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0\n'], ['Aristocats, The (1970)', '01-Jan-1970', '', 'http://us.imdb.com/M/title-exact?Aristocats,%20The%20(1970)', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0\n'], ['Terminator 2: Judgment Day (1991)', '01-Jan-1991', '', 'http://us.imdb.com/M/title-exact?Terminator%202:%20Judgment%20Day%20(1991)', '0', '1', '0', '0', '0

In [168]:
print(seen_by_user['23'])


[['Die Hard (1988)', '01-Jan-1988', '', 'http://us.imdb.com/M/title-exact?Die%20Hard%20(1988)', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0\n'], ['Mars Attacks! (1996)', '13-Dec-1996', '', 'http://us.imdb.com/M/title-exact?Mars%20Attacks!%20(1996)', '0', '1', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0\n'], ['Aliens (1986)', '01-Jan-1986', '', 'http://us.imdb.com/M/title-exact?Aliens%20(1986)', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '0\n'], ['Blade Runner (1982)', '01-Jan-1982', '', 'http://us.imdb.com/M/title-exact?Blade%20Runner%20(1982)', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0\n'], ['Brazil (1985)', '01-Jan-1985', '', 'http://us.imdb.com/M/title-exact?Brazil%20(1985)', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0\n'], ['Cinema Paradiso (1988)', '